**Notebook combining the 3 Dataframes `cities_df`, `airports_df` `weathers_df` `flights_df`**

# 0. Importing libraries 

In [1]:
# import libraries
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import json
from datetime import datetime, date, timedelta

# 1. Cities dataframe

1. Frankfurt
2. Berlin
3. Cologne
4. Munich
5. Hamburg

In [30]:
list_of_cities = ["Frankfurt", "Berlin", "Cologne", "Munich", "Hamburg"]

scooter_cities_df = pd.DataFrame(
    {"city_name": [],
     "country": [],
     "latitude": [],
     "longitude": [],
     "population": []
    }
)

for city in list_of_cities:
    #find url and store it in a variable
    url = "https://en.wikipedia.org/wiki/" + city
    #download html with a get request
    headers = {'Accept-Language': 'en-US,en;q=0.8'}
    response = requests.get(url, headers = headers)
    if response.status_code != 200: break  # 200 status code means OK!
    
    wiki_soup = BeautifulSoup(response.content, "html.parser")
    
    #extract name, country, latitude, longitude, population
    city_name = wiki_soup.select("span.mw-page-title-main")[0].getText()
    country_name = wiki_soup.select("table.infobox td.infobox-data")[0].getText()
    latitude = wiki_soup.select("span.latitude")[0].getText()
    longitude = wiki_soup.select("span.longitude")[0].getText()

    if wiki_soup.select_one('th.infobox-header:-soup-contains("Population")'):
        population = wiki_soup.select_one('th.infobox-header:-soup-contains("Population")').parent.find_next_sibling().find(text=re.compile(r'\d+'))
    
    #append information to the city_df
    city_df = pd.DataFrame(
        {"city_name": [city_name],
         "country": [country_name],
         "latitude": [latitude],
         "longitude": [longitude],
         "population": [population]
        # "population_density": [density]
        }
    )
    #scooter_cities_df = scooter_cities_df.append(city_df, ignore_index=True)
    scooter_cities_df = pd.concat([scooter_cities_df, city_df], ignore_index = True)
    
    # fixing latitude
    scooter_cities_df["latitude"] = scooter_cities_df["latitude"].str.split('″').str[0].str.replace('°', '.', regex=False).str.replace('′', '', regex=False)
    # fixing longitude
    scooter_cities_df["longitude"] = scooter_cities_df["longitude"].str.split('″').str[0].str.replace('°', '.', regex=False).str.replace('′', '', regex=False)
    # fixing population
    scooter_cities_df["population"] = scooter_cities_df["population"].str.replace(',', '', regex=False)
    # changing population datatype from object to integer
    #scooter_cities_df["population"] = scooter_cities_df["population"].astype(str).astype(int)
    
scooter_cities_df

,city_name,country,latitude,longitude,population
0,Frankfurt,Germany,50.0638,08.4056,764104
1,Berlin,Germany,52.3112,13.2418,3769495
2,Cologne,Germany,50.5611,6.5710,1083498
3,Munich,Germany,48.0815,11.3430,1488202
4,Hamburg,Germany,53.3300,10.0000,1845229


In [28]:
scooter_cities_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   city_name   1 non-null      object
 1   country     1 non-null      object
 2   latitude    1 non-null      object
 3   longitude   1 non-null      object
 4   population  1 non-null      object
dtypes: object(5)
memory usage: 168.0+ bytes


# 2. Airports dataframe

1. Frankfurt - EDDF - (50.033333, 8.570556)
2. Berlin - EDDB - (52.3667, 13.5020)
3. Cologne - EDDK - (50.8595, 7.1390)
4. Munich - EDDM - (48.362767, 11.76755)
5. Hamburg - EDDH - (53.62972, 9.991944)

In [32]:
def icao_airport_codes(latitudes, longitudes):

  #assert len(latitudes) == len(longitudes)
  
    list_for_airports_df = []

    for i in range(len(latitudes)):
        
        url = f"https://aerodatabox.p.rapidapi.com/airports/search/location/{latitudes[i]}/{longitudes[i]}/km/50/5"

   
        headers = {
            "X-RapidAPI-Key": "12c7bd34d2msha978e1852a9e04dp17b222jsnd5260f196022",
            "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
        }

        response = requests.request("GET", url, headers=headers)
        airports_resp = response.json()
        
        airports_df = pd.json_normalize(airports_resp["items"])[["icao", "name", "countryCode", "location.lat", "location.lon"]]
        airports_df = airports_df.rename(columns={"icao": "airport_icao", "name": "airport_name", "countryCode": "country_code", "location.lat": "latitude", "location.lon": "longitude"})
                
        list_for_airports_df.append(airports_df)
                    
    return pd.concat(list_for_airports_df, ignore_index=True)

airports_df = icao_airport_codes([50.033333, 52.3667, 50.8595, 48.362767, 53.62972], [8.570556, 13.5020, 7.1390, 11.76755, 9.991944])
airports_df

,airport_icao,airport_name,country_code,latitude,longitude
0,EDDF,Frankfurt-am-Main,DE,50.026400,8.543129
1,EDFE,"Egelsbach, Frankfurt-Egelsbach",DE,49.960000,8.645833
2,ETOU,"Wiesbaden, Wiesbaden Army Airfield",DE,50.049800,8.325400
3,EDFZ,"Mainz, Mainz-Finthen",DE,49.967500,8.147221
4,ETID,Hanau Army Air Field,DE,50.169200,8.961589
5,EDDB,"Berlin, Berlin Brandenburg",DE,52.351390,13.493889
6,EDDT,"Berlin, Berlin-Tegel",DE,52.559700,13.287699
7,EDAZ,"Trebbin, Schönhagen",DE,52.203610,13.156389
8,EDDK,"Cologne, Cologne Bonn",DE,50.865900,7.142739
9,EDKB,"Bonn, Bonn-Hangelar",DE,50.768890,7.163332


In [13]:
airports_df = icao_airport_codes([50.033333], [8.570556])
airports_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   airport_icao  5 non-null      object 
 1   airport_name  5 non-null      object 
 2   country_code  5 non-null      object 
 3   latitude      5 non-null      float64
 4   longitude     5 non-null      float64
dtypes: float64(2), object(3)
memory usage: 328.0+ bytes


# 3. Weathers dataframe

1. Frankfurt
2. Berlin
3. Cologne
4. Munich
5. Hamburg

In [3]:
def get_wea(cities):
    
    API_key = "7d5972a1b3270c831586122a13b70de5"
    
    now = datetime.now()
    
    wea_dict = {"city_id": [],
                "country_code": [],
                "forecast_time": [],
                "weather_outlook": [],
                "weather_detailed": [],
                "temperature": [],
                "feels_like_temperature": [],
                "humidity": [],
                "wind_speed": [],
                "risk_of_rain": [],
                "amount_of_rain": [],
                "amount_of_snow": [],
                "info_retrieved_at": []}
    
    for city in cities:
        url = f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric"
        response = requests.get(url)
        wea_resp = response.json()
        
        for wea in wea_resp["list"]:
            if city == "Frankfurt":
                wea_dict["city_id"].append(1)
            elif city == "Berlin":
                wea_dict["city_id"].append(2)
            elif city == "Cologne":
                wea_dict["city_id"].append(3)
            elif city == "Munich":
                wea_dict["city_id"].append(4)
            elif city == "Hamburg":
                wea_dict["city_id"].append(5)
            else:
                wea_dict["city_id"].append(0)
            
            wea_dict["country_code"].append(wea_resp["city"]["country"])
            wea_dict["forecast_time"].append(wea["dt_txt"])
            wea_dict["weather_outlook"].append(wea["weather"][0]["main"])
            wea_dict["weather_detailed"].append(wea["weather"][0]["description"])
            wea_dict["temperature"].append(wea["main"]["temp"])
            wea_dict["feels_like_temperature"].append(wea["main"]["feels_like"])
            wea_dict["humidity"].append(wea["main"]["humidity"])
            wea_dict["wind_speed"].append(wea["wind"]["speed"])
            wea_dict["risk_of_rain"].append(wea["pop"])
      
            try:
              wea_dict["amount_of_rain"].append(wea["rain"]["3h"])
            except:
              wea_dict["amount_of_rain"].append("0")
            try:
              wea_dict["amount_of_snow"].append(wea["snow"]["3h"])
            except:
              wea_dict["amount_of_snow"].append("0")
    
            wea_dict["info_retrieved_at"].append(now.strftime("%Y-%m-%d %H:%M:%S"))
    
    return pd.DataFrame(wea_dict)

weathers_df = get_wea(["Frankfurt", "Berlin", "Cologne", "Munich", "Hamburg"])
weathers_df

,city_id,country_code,forecast_time,weather_outlook,weather_detailed,temperature,feels_like_temperature,humidity,wind_speed,risk_of_rain,amount_of_rain,amount_of_snow,info_retrieved_at
0,1,DE,2022-11-07 12:00:00,Clouds,broken clouds,12.93,12.29,77,4.39,0.0,0,0,2022-11-07 10:25:21
1,1,DE,2022-11-07 15:00:00,Clouds,broken clouds,13.19,12.47,73,4.39,0.0,0,0,2022-11-07 10:25:21
2,1,DE,2022-11-07 18:00:00,Clouds,overcast clouds,11.95,11.21,77,3.10,0.0,0,0,2022-11-07 10:25:21
3,1,DE,2022-11-07 21:00:00,Clouds,overcast clouds,10.82,10.18,85,2.50,0.0,0,0,2022-11-07 10:25:21
4,1,DE,2022-11-08 00:00:00,Clouds,broken clouds,9.79,8.66,88,2.37,0.0,0,0,2022-11-07 10:25:21
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,5,DE,2022-11-11 21:00:00,Clouds,broken clouds,8.44,6.93,91,2.56,0.0,0,0,2022-11-07 10:25:21
196,5,DE,2022-11-12 00:00:00,Clouds,broken clouds,7.56,6.03,90,2.37,0.0,0,0,2022-11-07 10:25:21
197,5,DE,2022-11-12 03:00:00,Clouds,broken clouds,6.93,5.32,88,2.34,0.0,0,0,2022-11-07 10:25:21
198,5,DE,2022-11-12 06:00:00,Clouds,broken clouds,6.47,4.86,88,2.24,0.0,0,0,2022-11-07 10:25:21


**Change column `city` (which is the city name) to `city_id` (which will be the foreign key in this table in MySQL).**

In [35]:
weathers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   city_id                 200 non-null    int64  
 1   country_code            200 non-null    object 
 2   forecast_time           200 non-null    object 
 3   weather_outlook         200 non-null    object 
 4   weather_detailed        200 non-null    object 
 5   temperature             200 non-null    float64
 6   feels_like_temperature  200 non-null    float64
 7   humidity                200 non-null    int64  
 8   wind_speed              200 non-null    float64
 9   risk_of_rain            200 non-null    float64
 10  amount_of_rain          200 non-null    object 
 11  amount_of_snow          200 non-null    object 
 12  info_retrieved_at       200 non-null    object 
dtypes: float64(4), int64(2), object(7)
memory usage: 20.4+ KB


# 4. Flights dataframe

1. Frankfurt - EDDF
2. Berlin - EDDB
3. Cologne - EDDK
4. Munich - EDDM
5. Hamburg - EDDH

In [2]:
def tomorrows_flight_arrivals(icao_list):

    today = datetime.now().date()
    tomorrow = today + timedelta(days=1)
    
    list_for_arrivals_df = []
    
    for icao in icao_list:
        
        times = [["00:00","11:59"],["12:00","23:59"]]
        
        for time in times:
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"

            querystring = {"direction":"Arrival","withCancelled":"false"}

            headers = {
                "X-RapidAPI-Key": "12c7bd34d2msha978e1852a9e04dp17b222jsnd5260f196022",
                "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
                }

            response = requests.request("GET", url, headers=headers, params=querystring)
            flights_resp = response.json()
            
            arrivals_df = pd.json_normalize(flights_resp["arrivals"])[["number", "airline.name", "movement.scheduledTimeLocal", "movement.terminal", "movement.airport.name", "movement.airport.icao"]]
            arrivals_df = arrivals_df.rename(columns={"number": "flight_number", "airline.name": "airline", "movement.scheduledTimeLocal": "arrival_time", "movement.terminal": "arrival_terminal", "movement.airport.name": "departure_city", "movement.airport.icao": "departure_airport_icao"})
            arrivals_df["arrival_airport_icao"] = icao
            arrivals_df["data_retrieved_on"] = datetime.now().date()
            arrivals_df = arrivals_df[["arrival_airport_icao", "flight_number", "airline", "arrival_time", "arrival_terminal", "departure_city", "departure_airport_icao", "data_retrieved_on"]]
            
            # fixing arrival_time
            arrivals_df["arrival_time"] = arrivals_df["arrival_time"].str.split("+").str[0]
            
            list_for_arrivals_df.append(arrivals_df)
    
    return pd.concat(list_for_arrivals_df, ignore_index=True)

icao_list = ["EDDF", "EDDB", "EDDK", "EDDM", "EDDH"]

arrivals_df = tomorrows_flight_arrivals(icao_list)
arrivals_df

,arrival_airport_icao,flight_number,airline,arrival_time,arrival_terminal,departure_city,departure_airport_icao,data_retrieved_on
0,EDDF,DE 2017,Condor,2022-11-08 05:45,1,New York,KJFK,2022-11-07
1,EDDF,ET 706,Ethiopian,2022-11-08 05:25,1,Addis Ababa,HAAB,2022-11-07
2,EDDF,IR 1601,Iran Air,2022-11-08 05:20,1,Tehran,OIIE,2022-11-07
3,EDDF,LH 1327,Lufthansa,2022-11-08 05:40,1,Tunis,DTTA,2022-11-07
4,EDDF,LH 401,Lufthansa,2022-11-08 05:15,1,New York,KJFK,2022-11-07
...,...,...,...,...,...,...,...,...
2255,EDDH,LH 2086,Lufthansa,2022-11-08 22:05,2,Munich,EDDM,2022-11-07
2256,EDDH,LH 2088,Lufthansa,2022-11-08 22:35,2,Munich,EDDM,2022-11-07
2257,EDDH,LH 36,Lufthansa,2022-11-08 22:35,2,Frankfurt-am-Main,EDDF,2022-11-07
2258,EDDH,TK 1667,Turkish,2022-11-08 22:05,1,Istanbul,LTFM,2022-11-07


In [39]:
arrivals_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2338 entries, 0 to 2337
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   arrival_airport_icao    2338 non-null   object
 1   flight_number           2338 non-null   object
 2   airline                 2338 non-null   object
 3   arrival_time            2338 non-null   object
 4   arrival_terminal        2327 non-null   object
 5   departure_city          2338 non-null   object
 6   departure_airport_icao  2332 non-null   object
 7   data_retrieved_on       2338 non-null   object
dtypes: object(8)
memory usage: 146.2+ KB


# 5. Pushing data to MySQL

## 5.1 SQLAlchemy

In [ ]:
#!pip install sqlalchemy 
#import sqlalchemy

## 5.2 PyMySQL

In [ ]:
#!pip install PyMySQL

## 5.3 Specifying MySQL connection.

**Database `sql_scooters` has already been created on local mySQL server with corresponding tables...**

In [4]:
schema="sql_scooters"   # name of the database you want to use here
host="127.0.0.1"        # to connect to your local server
user="root"
password="Maxcavalera_75" # your password!!!!
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [31]:
scooter_cities_df.to_sql('cities',         # 'cities'-> table name;
              if_exists='append', # if_exists -> will create new table if doesn't exist, otherwise, 'append' - will append data to existing table;
              con=con,            # con-> connection string;
              index=False)        # index = False -> will not send index column to database

5

In [33]:
airports_df.to_sql('airports',         
              if_exists='append', 
              con=con,            
              index=False) 

17

In [5]:
weathers_df.to_sql('weathers',         
              if_exists='append', 
              con=con,            
              index=False)

200

In [6]:
arrivals_df.to_sql('flights',         
              if_exists='append', 
              con=con,            
              index=False)

2260